# Clases desbalanceadas

Conjunto de datos de entrenamiento con clases minoritarias, por lo que la información esta sesagada

## Podemos resolver el problema de diferentes formas: 

**Ajuste de Parámetros del modelo:** Ajustar la métrica del modelo para equilibrar a la clase minoritaria, dando un peso diferente durante el entrenamiento.

**Modificar el Dataset:**  Eliminar datos de la clase mayoritaria para reducirla.

**Muestras artificiales:** Crear muestras sintéticas utilizando algoritmos que intentan seguir la tendencia del grupo minoritario. 

**Ensamble de métodos:** Entrenar diversos modelos y entre todos obtener el resultado final.


https://www.aprendemachinelearning.com/clasificacion-con-datos-desbalanceados/

### Biblioteca: imbalanced-learn

https://pypi.org/project/imbalanced-learn/

$ pip install imbalanced-learn

# Credit Card Fraud Detection

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?select=creditcard.csv

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns 
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Importamos la clase de Regresión Lineal de scikit-learn
from sklearn.metrics import mean_squared_error, r2_score # error
from sklearn.metrics import classification_report

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek


from sklearn import metrics
from collections import Counter  

In [ ]:
mainpath = "../datasets/"
filename = 'creditcard.csv'
fullpath = os.path.join(mainpath, filename)
dataset= pd.read_csv (fullpath)
dataset

In [ ]:
print(pd.value_counts(dataset['Class'], sort = True))

In [ ]:
Y = dataset['Class']
X = dataset.drop('Class', axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7)

In [ ]:
def run_model(X_train, X_test, Y_train, Y_test):
    clf_base = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg")
    clf_base.fit(X_train, Y_train)
    return clf_base

In [ ]:
model = run_model(X_train, X_test, Y_train, Y_test)
Y_pred = model.predict(X_test)

In [ ]:
def show_result(Y_test, Y_pred):
    conf_matrix = metrics.confusion_matrix(Y_test, Y_pred)
    plt.figure(figsize=(5, 5))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap = 'Blues_r')
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print (classification_report(Y_test, Y_pred))

In [ ]:
show_result(Y_test, Y_pred)

### Modificar el desbalance con un peso

Utilizaremos un parámetro adicional en el modelo de Regresión logística en donde indicamos weight = “balanced” y con esto el algoritmo se encargará de equilibrar a la clase minoritaria durante el entrenamiento donde el peso asignado es:

\begin{equation}
w(j) = \frac{n}{K * n(j)}
\end{equation}

donde $n$ es el numero de datos, $K$ es el total de clases y $n(j)$ es el número de datos de cada clase

In [ ]:
def run_model_balanced(X_train, X_test, Y_train, Y_test):
   # clf = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight={0: 0.2,1: 0.8})
    clf = LogisticRegression(C=1.0,penalty='l2',random_state=1,solver="newton-cg",class_weight="balanced")
    clf.fit(X_train, Y_train)
    return clf
 

In [ ]:

model = run_model_balanced(X_train, X_test, Y_train, Y_test)
Y_pred = model.predict(X_test)
show_result(Y_test, Y_pred)

### NearMiss 

Es un algoritmo que reduce la clase mayoritaria que utiliza un algoritmo similar al k-nearest neighbor para ir seleccionando cuales eliminar.


In [ ]:
us = NearMiss()
X_train_res, Y_train_res = us.fit_resample(X_train, Y_train)
 
print ("before resampling {}".format(Counter(Y_train)))
print ("after resampling {}".format(Counter(Y_train_res)))
 
model = run_model(X_train_res, X_test, Y_train_res, Y_test)
Y_pred = model.predict(X_test)
show_result(Y_test, Y_pred)

### RandomOverSampler

Crea una muestras nuevas “sintéticas” de la clase minoritaria 

In [ ]:
os =  RandomOverSampler()
X_train_res, Y_train_res = os.fit_resample(X_train, Y_train)
 
print ("before resampling {}".format(Counter(Y_train)))
print ("after resampling {}".format(Counter(Y_train_res)))
 
model = run_model(X_train_res, X_test, Y_train_res, Y_test)
Y_pred = model.predict(X_test)
show_result(Y_test, Y_pred)

### SMOTE (Synthetic Minority Over-sampling Technique)

SMOTE es un método de sobremuestreo que crea una muestras sintéticas (no duplicadas) de la clase minoritaria. hasta que sea igual a la clase mayoritaria. SMOTE hace esto seleccionando registros similares y alterando ese registro una columna a la vez aleatoriamente.

In [ ]:
os_us = SMOTE()
X_train_res, Y_train_res = os_us.fit_resample(X_train, Y_train)
 
print ("before resampling {}".format(Counter(Y_train)))
print ("after resampling {}".format(Counter(Y_train_res)))
 
model = run_model(X_train_res, X_test, Y_train_res, Y_test)
Y_pred = model.predict(X_test)
show_result(Y_test, Y_pred)

### SMOTETomek

Consiste en aplicar en simultáneo un algoritmo de subsampling y otro de oversampling: SMOTE para oversampling: busca puntos vecinos cercanos y agrega puntos “en linea recta” entre ellos y Tomek para undersampling que quita los de distinta clase que sean vecinos cercanos.

In [ ]:
os_us = SMOTETomek()
X_train_res, Y_train_res = os_us.fit_resample(X_train, Y_train)
 
print ("before resampling {}".format(Counter(Y_train)))
print ("after resampling {}".format(Counter(Y_train_res)))
 
model = run_model(X_train_res, X_test, Y_train_res, Y_test)
pred_y = model.predict(X_test)
show_result(Y_test, Y_pred)

### Ensamble de Modelos con Balanceo

El ensamble de modelos es una técnica usada para reducir la varianza de las predicciones a través de la combinación de los resultados de varios clasificadores. Además de esto, se aborda el problema de los datos desequilibrados mediante el uso de submuestreo aleatorio para equilibrar la distribución de clases en cada subconjunto. Esto ayuda a reducir el sesgo hacia la clase mayoritaria y mejorar el desempeño de la clase minoritaria.


In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
 
bbc.fit(X_train, Y_train)
Y_pred = bbc.predict(X_test)
show_result(Y_test, Y_pred)